In [ ]:
#hide
#default_exp tests
from nbdev import *
%load_ext autoreload
%autoreload 2

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


[autoreload of drt.data_model failed: Traceback (most recent call last):
  File "/opt/miniconda3/envs/drt/lib/python3.8/site-packages/IPython/extensions/autoreload.py", line 245, in check
    superreload(m, reload, self.old_objects)
  File "/opt/miniconda3/envs/drt/lib/python3.8/site-packages/IPython/extensions/autoreload.py", line 410, in superreload
    update_generic(old_obj, new_obj)
  File "/opt/miniconda3/envs/drt/lib/python3.8/site-packages/IPython/extensions/autoreload.py", line 347, in update_generic
    update(a, b)
  File "/opt/miniconda3/envs/drt/lib/python3.8/site-packages/IPython/extensions/autoreload.py", line 292, in update_class
    if (old_obj == new_obj) is True:
  File "/opt/miniconda3/envs/drt/lib/python3.8/site-packages/sqlalchemy/sql/operators.py", line 365, in __eq__
    return self.operate(eq, other)
  File "/opt/miniconda3/envs/drt/lib/python3.8/site-packages/sqlalchemy/orm/attributes.py", line 226, in operate
    return op(self.comparator, *other, **kwargs)
 

# Tests

> Tests to see if stuff works and to illustrate a few points 

In [ ]:
#hide
#export
from nbdev.showdoc import *
from drt.create_environment import *
from drt.environment import DataIntakeEnv
from drt.registration import register_all
from drt.receipt import sync_data_folder
from drt.verification import check_datagroup
import drt.data_model as dm
from pathlib import Path
import shutil
import drt
import re
from zipfile import ZipFile
try:
    import importlib.resources as pkg_resources
except ImportError:
    # Try backported to PY<37 `importlib_resources`.
    import importlib_resources as pkg_resources
    

In [ ]:
#export
def create_test_folder(test_path:Path):
    """
    Create a folder in a location with dummy data ready to be registered
    """
    if test_path.exists() and test_path.is_dir():
        shutil.rmtree(test_path)
    create_environment(test_path)
    # TODO : Turn this into a zip extraction to avoid interference

    (test_path/'01_Delivery').rmdir()
    (test_path/'02_RAW').rmdir()
    (test_path/'03_Datasets').rmdir()

    with pkg_resources.path(drt, 'test_files') as test:
        with ZipFile(test/'zipped_test.zip', 'r') as zipObj:
            zipObj.extractall(test_path)
        
        # shutil.copytree((test/'01_Delivery'), (test_path / '01_Delivery'),dirs_exist_ok=True)
        # shutil.copytree((test/'02_RAW'), (test_path / '02_RAW'),dirs_exist_ok=True)
        # shutil.copytree((test/'03_Datasets'), (test_path / '03_Datasets'),dirs_exist_ok=True)
    env = DataIntakeEnv(test_path / '.config' / 'config.ini')

    register_all(env)

    for data_folder in test_path.iterdir():
        if data_folder.is_dir():
            for data_group in data_folder.iterdir():
                if data_group.name[0] not in ['.','_']:
                    if data_folder.name == '01_Delivery':
                        if (data_group/'receipt.rst').exists():
                            with open(data_group/'receipt.rst', 'rt') as recpt:
                                text = recpt.read()
                                text = re.sub('(.*\n:Date Received: )None(.*)', '\\1 2020-01-04\\2', text, re.MULTILINE)
                                text = re.sub('(.*\n:Received from: )None(.*)', '\\1Dummy Recepient\\2',text, re.MULTILINE)
                                text = re.sub('(.*\nDescription:\n-+\n)None(.*)', '\\1Lorem ipsum Dolor sit Amet\\2', text, re.MULTILINE)
                            with open(data_group/'receipt.rst', 'wt') as recpt:
                                recpt.write(text)
                    else:
                        if (data_group/'receipt.rst').exists():
                            with open(data_group/'receipt.rst', 'rt') as recpt:
                                text = recpt.read()
                                text = re.sub('(.*\nDescription:\n-+\n)None(.*)', '\\1Lorem ipsum Dolor sit Amet\\2', text, re.MULTILINE)
                                text = re.sub('(.*\nReport\n-+\n)None(\n.*)', '\\1Dummy HTML Report\\2', text, re.MULTILINE)
                            with open(data_group/'receipt.rst', 'wt') as recpt:
                                recpt.write(text)

    [sync_data_folder(env,group_type ) for group_type in [dm.Delivery, dm.Raw_Data, dm.Dataset] ]
    



In [ ]:
test_path = Path('test_env')
create_test_folder(test_path)

  0%|          | 0/1 [00:00<?, ?files/s]

  0%|          | 0/1 [00:00<?, ?files/s]

  0%|          | 0/6 [00:00<?, ?files/s]

  0%|          | 0/2 [00:00<?, ?files/s]

  0%|          | 0/4 [00:00<?, ?files/s]

...

  0%|          | 0/3 [00:00<?, ?files/s]

  0%|          | 0/3 [00:00<?, ?files/s]

  0%|          | 0/3 [00:00<?, ?files/s]

  0%|          | 0/3 [00:00<?, ?files/s]

Writing receipt to test_env/01_Delivery/2020_01_10_iris_csv
Writing receipt to test_env/01_Delivery/2020_01_11_dump
Writing receipt to test_env/01_Delivery/2021_01_19_Iris
Writing receipt to test_env/01_Delivery/2020_01_09_near_earth_json
Writing receipt to test_env/01_Delivery/2020_01_12_missing_source
Writing receipt to test_env/01_Delivery/2020_01_08_video_files
Writing receipt to test_env/01_Delivery/2021_01_19_Mnist
Writing receipt to test_env/01_Delivery/2020_01_10_measurements_excel
Writing receipt to test_env/02_RAW/2020_02_03_iris_versicolor
Writing receipt to test_env/02_RAW/2020_02_02_iris_versicolor
Writing receipt to test_env/02_RAW/2020_02_03_missing_source
Writing receipt to test_env/02_RAW/2020_02_01_rick_roll_videos
Writing receipt to test_env/02_RAW/2020_02_02_metal_measurements
Writing receipt to test_env/02_RAW/2020_02_01_near_earth_objects
Writing receipt to test_env/02_RAW/2020_02_03_netflix_titles
Writing receipt to test_env/02_RAW/2020_02_02_temperature_mass_mea

In [ ]:
text = """

-------------
:Files Last Changed: 2021-02-10 20:20:18.899966
:File Size: 1244762
:Number of Files: 3
:Hash: 975bd61edb1adec7b61b8fb459de04e775d286bcc5cd19a4be67c74c04044e46

Report
------
None

"""
re.sub('(.*\nReport\n-+\n)None(\n.*)', '\\1Dummy HTML Report\\2', text, re.MULTILINE)

'\n\n-------------\n:Files Last Changed: 2021-02-10 20:20:18.899966\n:File Size: 1244762\n:Number of Files: 3\n:Hash: 975bd61edb1adec7b61b8fb459de04e775d286bcc5cd19a4be67c74c04044e46\n\nReport\n------\nDummy HTML Report\n\n'

In [ ]:
env = DataIntakeEnv(test_path / '.config' / 'config.ini')

[sync_data_folder(env,group_type ) for group_type in [dm.Delivery, dm.Raw_Data, dm.Dataset] ]

Writing receipt to test_env/01_Delivery/2020_01_10_iris_csv
Writing receipt to test_env/01_Delivery/2020_01_11_dump
Writing receipt to test_env/01_Delivery/2021_01_19_Iris
Writing receipt to test_env/01_Delivery/2020_01_09_near_earth_json
Writing receipt to test_env/01_Delivery/2020_01_12_missing_source
Writing receipt to test_env/01_Delivery/2020_01_08_video_files
Writing receipt to test_env/01_Delivery/2021_01_19_Mnist
Writing receipt to test_env/01_Delivery/2020_01_10_measurements_excel
Writing receipt to test_env/02_RAW/2020_02_03_iris_versicolor
Writing receipt to test_env/02_RAW/2020_02_02_iris_versicolor
Writing receipt to test_env/02_RAW/2020_02_03_missing_source
Writing receipt to test_env/02_RAW/2020_02_01_rick_roll_videos
Writing receipt to test_env/02_RAW/2020_02_02_metal_measurements
Writing receipt to test_env/02_RAW/2020_02_01_near_earth_objects
Writing receipt to test_env/02_RAW/2020_02_03_netflix_titles
Writing receipt to test_env/02_RAW/2020_02_02_temperature_mass_mea

[None, None, None]

In [ ]:
[check_datagroup(env, group_type, light=False) for group_type in [dm.Delivery, dm.Raw_Data, dm.Dataset]]

  0%|          | 0/9 [00:00<?, ?data group/s]

  0%|          | 0/1 [00:00<?, ?files/s]

  0%|          | 0/4 [00:00<?, ?files/s]

  0%|          | 0/1 [00:00<?, ?files/s]

  0%|          | 0/1 [00:00<?, ?files/s]

...

  0%|          | 0/3 [00:00<?, ?data group/s]

  0%|          | 0/3 [00:00<?, ?files/s]

  0%|          | 0/3 [00:00<?, ?files/s]

  0%|          | 0/3 [00:00<?, ?files/s]

[['[DLV] 2021_01_19_Iris no data extracted',
  '[DLV] Registered data missing files or empty folder at test_env/01_Delivery/2020_01_13_no structure',
  '[DLV] No Data registered for: 2020_01_13_no structure',
  '[DLV] 2021_01_19_Mnist no data extracted'],
 [],
 []]

In [ ]:
shutil.rmtree(test_path)

In [ ]:
#hide
from nbdev.export import notebook2script; notebook2script()

Converted 00_tests.ipynb.
Converted 01_cli.ipynb.
Converted 02_create_environment.ipynb.
Converted 03_data_model.ipynb.
Converted 04_environment.ipynb.
Converted 05_receipt.ipynb.
Converted 06_registration.ipynb.
Converted 07_verification.ipynb.
Converted 99_utils.ipynb.
Converted index.ipynb.
